### 配置文件验证

In [ ]:
from omegaconf import OmegaConf
hparams = OmegaConf.load('config.yaml') # 读取配置文件

In [ ]:
hparams.pro_type_file is None

In [ ]:
from modelModule.model5 import VAE5
import torch
model = VAE5(dim=16, pro_types=pro_types)

### 得到数据集相关属性

In [ ]:
import pickle
import pandas as pd
data_pd = pd.read_csv('Letter_train.csv')
col_names = list(data_pd)
# df[col_name].value_counts()
pro_types = [('discrete',16) for _ in col_names]
# with open('Letter_pro_type.pkl', 'wb') as f:
#     pickle.dump(pro_types, f)

In [ ]:
a = 0
a is not None

In [ ]:
import pickle
pro_types = pickle.load(open('Credit_pro_type.pkl', 'rb'))
pro_types

### dataloader验证

In [ ]:
import torch
from dataModule.dataset1 import FlatDataset
from torch.utils.data import DataLoader

In [ ]:
dataset = FlatDataset('Letter_train.csv', 
                    pro_type_file='Letter_pro_type.pkl')


In [ ]:
train_dataloader = DataLoader(dataset, 
                             batch_size=10, 
                             collate_fn=dataset.collater, 
                             shuffle=True,
                             num_workers=2)
batch = None
for i in train_dataloader:
    batch = i
    break

In [ ]:
batch['normal_data']

In [ ]:
batch["miss_data"].long()

In [ ]:
batch["src_data"]

In [ ]:
test_data = imputed_data * global_max + global_min # 广播效应

In [ ]:
a = torch.randint(0,10, size=(5,4,3)).float()
print(a)
f = torch.nn.Softmax(dim=1)
f(a)


In [ ]:
a = torch.randint(0,10,size=(5,12,4)).float()
print(a)
model = torch.nn.Linear(4,10)
model(a[:,1,:]).shape

In [ ]:
a[:,0].reshape(-1,1)

In [ ]:
model(a[:,0].reshape(-1,1,1)).permute(0, 2, 1).shape

In [ ]:
isinstance(model, torch.nn.Embedding)

In [ ]:
a = torch.randint(0,2,size=(5, 3))
b = torch.nn.Parameter(torch.randn(1, 4))
print(b)
print(a)

miss_matric = a.unsqueeze (-1).expand(5,3,4)
print(miss_matric)
miss_matric*b

In [ ]:
a = torch.randint(0,10,size=(5,3)).float()
model = torch.nn.Conv1d(in_channels=1, out_channels=8, kernel_size=1, stride=1)
print(a)
embedding_out = torch.tensor([])
embedding_out = torch.cat((embedding_out, model(a[:,0].reshape(-1,1,1)).permute(0, 2, 1)), dim = 1)
embedding_out = torch.cat((embedding_out, model(a[:,1].reshape(-1,1,1)).permute(0, 2, 1)), dim = 1)
print(embedding_out.shape)
embedding_out

In [ ]:
b.shape

### 模型5精度测定

In [1]:
from dataModule.dataset1 import FlatDataset
# from modelModule.model1 import VAE
# from modelModule.model2 import VAE2
# from modelModule.model4 import VAE4
from modelModule.model5 import VAE5
from torch.utils.data import DataLoader
from utils import result_show, get_missing
import torch
import numpy as np
import pickle

In [2]:
## 模型参数载入
model = VAE5(dim=23, pro_types=pickle.load(open('Credit_pro_type.pkl', 'rb'))) # nhead=3
checkpoint_path = r'模型参数保存/Credit_Card_model5_norm_way_minmax_norm_/version_1/checkpoints/epoch=12-step=1222.ckpt'    
data_norm = 'minmax_norm'  # 数据集采用正则化方式minmax_norm,mean_norm
train_csv = r'Credit_Card_标签置换后train.csv'
test_csv = r'Credit_Card_标签置换后test.csv'

checkpoint = torch.load(checkpoint_path)
model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
model.eval()
## 载入训练集的均值(最小)方差(最大)
train_dataset = FlatDataset(train_csv, data_norm=data_norm, pro_type_file='Credit_pro_type.pkl')
model.get_global_min_max(dataset=train_dataset)
print('载入成功')

载入成功


In [3]:
test_dataset = FlatDataset(test_csv, data_norm=data_norm, pro_type_file='Credit_pro_type.pkl')
data_size = 10
src_data = np.stack([test_dataset[index]['src_data'] for index in range(data_size)], axis=0)
normal_data = np.stack([test_dataset[index]['global_normal_data'] for index in range(data_size)], axis=0)
portion_normal_data = np.stack([test_dataset[index]['portion_normal_data'] for index in range(data_size)], axis=0)
_, miss_matrix = get_missing(src_data, 0.3) # 得到缺失矩阵
input = portion_normal_data.copy()
input[miss_matrix == 0] = np.nan

In [5]:
imputed_data, output = model.inference(input, miss_matrix) # inference输出的结果已经重构后的结果
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()

In [7]:
print(src_data[0])
print(imputed_data[0])

[2.0000e+04 0.0000e+00 1.0000e+00 2.0000e+00 3.3000e+01 3.0000e+00
 4.0000e+00 4.0000e+00 4.0000e+00 3.0000e+00 3.0000e+00 1.7971e+04
 1.7399e+04 1.9057e+04 1.8453e+04 1.9755e+04 1.9288e+04 0.0000e+00
 2.2600e+03 0.0000e+00 1.6000e+03 0.0000e+00 6.4400e+02]
[ 2.0929391e+04  8.0085373e-01  8.4647548e-01  2.0833080e+00
  2.2022539e+01  8.0249989e-01  3.1824083e+00  5.1908331e+00
  6.1104929e-01  1.2246548e+00  4.7948027e+00 -1.4335464e+05
  3.8314727e+04 -6.9132812e+03  1.3795653e+05  3.2394303e+05
 -2.7688888e+05  1.8394273e+04  6.9379914e+04  3.0167070e+04
  4.4772871e+04  1.6334787e+04  2.6837441e+05]


In [ ]:
imputed_data[1]

In [ ]:
result_show(src_data, imputed_data, miss_matrix)

In [ ]:
imputed_data

In [ ]:
imputed_data.shape

In [ ]:
# test_dataset = FlatDataset('Letter_test.csv', data_norm='mean_norm') # minmax_norm
# index = 102
# src_data, M_matrix, miss_data = test_dataset[index]['src_data'].reshape(1,-1), test_dataset[index]['global_miss_matrix'].reshape(1,-1), test_dataset[index]['global_miss_data'].reshape(1,-1)
# src_data = test_dataset[index]['src_data'].reshape(1,-1)
# normal_data = test_dataset[index]['global_normal_data'].reshape(1,-1)
# miss_data, M_matrix = get_missing(normal_data, 0.3) # 产生掩码数据，并采用随机数填补

# imputed_data, _, _ = model(torch.from_numpy(miss_data).float(), torch.from_numpy(M_matrix).float()) 
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy() # 得到最终插补结果,数据由[0,1]范围内
# res_imputed_data = np.around(restore_data(imputed_data, test_dataset.Max_Val, test_dataset.Min_Val))
# result_show(src_data, res_imputed_data, M_matrix)

In [ ]:
# batch测试
# src_data, miss_data, M_matrix = batch['src_data'], batch['miss_data'], batch['miss_matrix']
# normal_data = batch['normal_data']
# imputed_data, _, _ = model(miss_data, M_matrix)
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()
# res_imputed_data = np.around(restore_data(imputed_data, dataset.Max_Val, dataset.Min_Val))